#Clean data

## ขั้นตอนที่1 Import Data

โดยการโหลดข้อมูลจากทาง Google drive โดยไฟล์มีชื่อว่า "data.csv"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/data.csv")

Mounted at /content/drive


##ขั้นตอนที่2 Import Library



In [ ]:
## import for data
import json
import pandas as pd
import numpy as np
## import for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## import for processing
import re
import nltk
## import for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## import for word embedding
import gensim
import gensim.downloader as gensim_api
## import for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K


แสดงข้อมูลทั้งหมดอออกมาดูว่ามีข้อมูลเป็นลักษณะเช่นไร

In [ ]:
df

,Teacher,Title/Subject,Year,Source title,Cited by,Link,Abstract,Author Keywords,Index Keywords,Document Type,EID,Unnamed: 11
0,tnt,An association rule mining approach to discove...,2021,International Journal of Knowledge and System...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"In the digital age, social media technology ha...",Association rule mining; Demand and supply ana...,NaN,Article,2-s2.0-85104037866,NaN
1,pps,Improvement of Difficulty Parameter Equating b...,2021,KST 2021 - 2021 13th International Conference ...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"In automated test construction, one of the rem...",distribution; Item Response Theory (IRT); Line...,Mean square error; Automated test construction...,Conference Paper,2-s2.0-85105890877,NaN
2,pps,Data Preparation for Reducing Computational Ti...,2021,KST 2021 - 2021 13th International Conference ...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,Training model in deep learning on action reco...,Action recognition; Computational cost; Deep l...,Deep learning; Action recognition; Computation...,Conference Paper,2-s2.0-85105869338,NaN
3,ppr,Overhead study of telegraf as a real-time moni...,2020,JCSSE 2020 - 17th International Joint Conferen...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,Large-scale distributed systems have become an...,Datacenter; Experimental study; IoT; Monitorin...,NaN,Conference Paper,2-s2.0-85098488755,NaN
4,ssr,Method Evaluation for Software Testability on ...,2020,2020 59th Annual Conference of the Society of ...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"In software testability, there are various sof...",Software Metrics tools; Software System Evalua...,Artificial intelligence; Budget control; Compu...,Conference Paper,2-s2.0-85096361247,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
599,lpp,INSTRUCTIONAL DESIGN AND DEVELOPMENT OF INTERA...,2018,NaN,NaN,NaN,Principles of and research in interactive mult...,NaN,NaN,Course,NaN,NaN
600,lpp,SELECTED TOPICS IN HUMAN-COMPUTER INTERACTION,2018,NaN,NaN,NaN,Seminar on current research and development in...,NaN,NaN,Course,NaN,NaN
601,wdp,NETWORK DESIGN AND MANAGEMENT,2018,NaN,NaN,NaN,Topics in computer network design and manageme...,NaN,NaN,Course,NaN,NaN
602,ppr,SELECTED TOPICS IN DATA SCIENCE,2018,NaN,NaN,NaN,The purpose of Selected Topics in Data Science...,NaN,NaN,Course,NaN,NaN


## ขั้นตอนที่3 Dropping Columns in a DataFrame

จะเห็นได้ว่ามีบางคอลัมน์ที่ไม่ได้ใช้ประโยชน์ โดยจะ drop คอลัมน์ ดังต่อไปนี้ Source title ,cited by,year,  Link ,	Document Type ,	EID และ Unnamed: 11 เนื่องจากเป็นข้อมูลที่ไม่ใช้

In [ ]:
df.drop(['Source title', 'Link','Year','Cited by','Document Type', 'EID','Unnamed: 11'] , axis='columns', inplace=True)

In [ ]:
df

,Teacher,Title/Subject,Abstract,Author Keywords,Index Keywords
0,tnt,An association rule mining approach to discove...,"In the digital age, social media technology ha...",Association rule mining; Demand and supply ana...,NaN
1,pps,Improvement of Difficulty Parameter Equating b...,"In automated test construction, one of the rem...",distribution; Item Response Theory (IRT); Line...,Mean square error; Automated test construction...
2,pps,Data Preparation for Reducing Computational Ti...,Training model in deep learning on action reco...,Action recognition; Computational cost; Deep l...,Deep learning; Action recognition; Computation...
3,ppr,Overhead study of telegraf as a real-time moni...,Large-scale distributed systems have become an...,Datacenter; Experimental study; IoT; Monitorin...,NaN
4,ssr,Method Evaluation for Software Testability on ...,"In software testability, there are various sof...",Software Metrics tools; Software System Evalua...,Artificial intelligence; Budget control; Compu...
...,...,...,...,...,...
599,lpp,INSTRUCTIONAL DESIGN AND DEVELOPMENT OF INTERA...,Principles of and research in interactive mult...,NaN,NaN
600,lpp,SELECTED TOPICS IN HUMAN-COMPUTER INTERACTION,Seminar on current research and development in...,NaN,NaN
601,wdp,NETWORK DESIGN AND MANAGEMENT,Topics in computer network design and manageme...,NaN,NaN
602,ppr,SELECTED TOPICS IN DATA SCIENCE,The purpose of Selected Topics in Data Science...,NaN,NaN


In [ ]:
#เปลี่ยนชื่อcolumn ให้เข้าใจได้ง่ายขึ้น
df = df.rename(columns={'Title/Subject': 'Title'})

ตรวจสอบว่าในแต่ละคอลัมน์มีข้อมูลที่ขาดหายไปจำนวนเท่าไหร่

In [ ]:
df.isnull().sum()

Teacher             54
Title               14
Abstract            13
Author Keywords    371
Index Keywords     410
dtype: int64

## ขั้นตอนที่4 Check and create new columns

In [ ]:
#drop ข้อมูลของอาจารย์ผู้สอนที่เป็น null ออกเนื่องจากเป็นอาจารย์ที่ไม่ได้อยู่ในภาควิชา
#drop ข้อมูลที่ไม่มีชื่อเรื่องออก
df = df.dropna(how='all',subset=['Teacher'])
df = df.dropna(how='all',subset=['Title'])
#เติมข้อมูลที่เป็น null ด้วยช่องว่างเพื่อเวลานำไปใช้งานต่อไป
df['Author Keywords'].fillna(" ", inplace=True)
df['Abstract'].fillna(" ", inplace=True)
df['Index Keywords'].fillna(" ", inplace=True)
df.isnull().sum()

Teacher            0
Title              0
Abstract           0
Author Keywords    0
Index Keywords     0
dtype: int64

เพิ่มคอลัมน์ชื่อว่า "add" โดย นำคอลัมน์ต่อไปนี้มารวมข้อความกัน Title , Abstract , Index Keywords , Author Keywords

In [ ]:
df["add"] =  df["Title"] +df["Abstract"] + df["Index Keywords"] +  df["Author Keywords"]

##ขั้นตอนที่5 Preprocessing

โดยสร้างฟังก์ชัน ชื่อว่า utils_preprocess_text

clean - แปลงตัวอักษรให้เป็นตัวอักษรพิมพ์เล็กทั้งหมด ลบเครื่องหมายวรรคตอนและอักขระออก

Tokenize - แปลง string เป็น list

Stemming - ลบ  -ing, -ly, ฯลฯ 

Lemmatisation - แปลงคำศัพท์ เป็น รากคำศัพท์


In [ ]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

เรียกใช้ โมดูลในภาษาไพทอนที่ช่วยในการประมวลภาษาธรรมชาติ ( Natural Language Toolkit )

โดยการดาวน์โหลด "stopword" เป็นคำที่ไม่มีความหมายที่ชัดเจน ดังต่อไปนี้

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

สร้างคอลัมน์ใหม่ที่ชื่อว่า "text_Abstract" โดย เรียกใช้ฟังก์ชัน utils_preprocess_text


In [ ]:
df["text_clean"] = df["add"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))

In [ ]:
#print ข้อมูลออกมาดูจะพบว่าข้อมูล column text_clean สามารถนำไปวิเคราะห์ต่อได้
df

,Teacher,Title,Abstract,Author Keywords,Index Keywords,add,text_clean
0,tnt,An association rule mining approach to discove...,"In the digital age, social media technology ha...",Association rule mining; Demand and supply ana...,,An association rule mining approach to discove...,association rule mining approach discover dema...
1,pps,Improvement of Difficulty Parameter Equating b...,"In automated test construction, one of the rem...",distribution; Item Response Theory (IRT); Line...,Mean square error; Automated test construction...,Improvement of Difficulty Parameter Equating b...,improvement difficulty parameter equating base...
2,pps,Data Preparation for Reducing Computational Ti...,Training model in deep learning on action reco...,Action recognition; Computational cost; Deep l...,Deep learning; Action recognition; Computation...,Data Preparation for Reducing Computational Ti...,data preparation reducing computational time t...
3,ppr,Overhead study of telegraf as a real-time moni...,Large-scale distributed systems have become an...,Datacenter; Experimental study; IoT; Monitorin...,,Overhead study of telegraf as a real-time moni...,overhead study telegraf realtime monitoring ag...
4,ssr,Method Evaluation for Software Testability on ...,"In software testability, there are various sof...",Software Metrics tools; Software System Evalua...,Artificial intelligence; Budget control; Compu...,Method Evaluation for Software Testability on ...,method evaluation software testability object ...
...,...,...,...,...,...,...,...
599,lpp,INSTRUCTIONAL DESIGN AND DEVELOPMENT OF INTERA...,Principles of and research in interactive mult...,,,INSTRUCTIONAL DESIGN AND DEVELOPMENT OF INTERA...,instructional design development interactive m...
600,lpp,SELECTED TOPICS IN HUMAN-COMPUTER INTERACTION,Seminar on current research and development in...,,,SELECTED TOPICS IN HUMAN-COMPUTER INTERACTIONS...,selected topic humancomputer interactionsemina...
601,wdp,NETWORK DESIGN AND MANAGEMENT,Topics in computer network design and manageme...,,,NETWORK DESIGN AND MANAGEMENTTopics in compute...,network design managementtopics computer netwo...
602,ppr,SELECTED TOPICS IN DATA SCIENCE,The purpose of Selected Topics in Data Science...,,,SELECTED TOPICS IN DATA SCIENCEThe purpose of ...,selected topic data sciencethe purpose selecte...


In [ ]:
#download file csv เพื่อนำไปวิเคราะห์ต่อ
from google.colab import files
df.to_csv('dataclean.csv') 
files.download('dataclean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>